<a href="https://colab.research.google.com/github/JacobFV/DesparadosAEYE/blob/main/ConversationAIipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversation AI

This jupyter notebook is used to build and train our conversation AI. You can run it by clicking the "Open in Colab" button above

In [3]:
%tensorflow_version 2.x
import os
import re

import tensorflow as tf
import tensorflow_datasets as tfds

keras = tf.keras
tfkl = keras.layers

## Get data

This definitely needs more work

In [4]:
path_to_zip = tf.keras.utils.get_file(
    'cornell_movie_dialogs.zip',
    origin=
    'http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
    extract=True)

path_to_dataset = os.path.join(
    os.path.dirname(path_to_zip), "cornell movie-dialogs corpus")

path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
path_to_movie_conversations = os.path.join(path_to_dataset,
                                           'movie_conversations.txt')

In [11]:
# Maximum number of samples to preprocess
MAX_SAMPLES = 50000

def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
  sentence = sentence.strip()
  # adding a start and an end token to the sentence
  return sentence


def load_conversations():
  # dictionary of line id to text
  id2line = {}
  with open(path_to_movie_lines, errors='ignore') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    id2line[parts[0]] = parts[4]

  inputs, outputs = [], []
  with open(path_to_movie_conversations, 'r') as file:
    lines = file.readlines()
  for line in lines:
    parts = line.replace('\n', '').split(' +++$+++ ')
    # get conversation in a list of line ID
    conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]
    for i in range(len(conversation) - 1):
      inputs.append(preprocess_sentence(id2line[conversation[i]]))
      outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))
      if len(inputs) >= MAX_SAMPLES:
        return inputs, outputs
  return inputs, outputs


questions, answers = load_conversations()

## Build model

In [13]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.4MB/s 
     |████████████████████████████████| 3.2MB 16.2MB/s 
     |████████████████████████████████| 890kB 32.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=16d79382548c5324a82355bce4d79307876de8c1ac3fcfaa68e8417933bb3d8b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [14]:
import transformers

In [42]:
gpt = transformers.TFOpenAIGPTLMHeadModel.from_pretrained('openai-gpt')

All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


In [30]:
tokenizer = transformers.OpenAIGPTTokenizer.from_pretrained('openai-gpt')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [44]:
enc = tokenizer.encode("Hello my name is Jacob", return_tensors="tf")
enc

<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[3570,  547, 1362,  544, 5365]], dtype=int32)>

In [45]:
gpt.generate(enc)

<tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[ 3570,   547,  1362,   544,  5365,   239,   244, 40477,   244,
         5365,   240,   244,   249,   603,   240,   244,   249,   719,
         1458,   240]], dtype=int32)>